In [4]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional, Concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Custom Loss for Regression
def custom_loss(y_true, y_pred):
    error = tf.abs(y_true - y_pred)
    weight = tf.where(error > 0.05, 5.0, 1.0)
    weighted_mse = tf.reduce_mean(weight * tf.square(y_true - y_pred))
    direction = tf.reduce_mean(tf.sign(y_true[1:] - y_true[:-1]) * tf.sign(y_pred[1:] - y_pred[:-1]))
    drop_penalty = tf.reduce_mean(tf.where(y_true < y_pred, 15.0 * tf.square(y_true - y_pred), 0.0))
    actual_vol = tf.math.reduce_variance(y_true)
    pred_vol = tf.math.reduce_variance(y_pred)
    vol_penalty = tf.square(actual_vol - pred_vol)
    return weighted_mse - 0.1 * direction + 0.5 * drop_penalty + 0.2 * vol_penalty

# Focal Loss for Classification
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Updated LSTM Model with GRU and Wavelet Branch
def create_lstm_model(units, dropout_rate, sequence_length, num_features, num_wavelet_features=5):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    main_features = inputs[:, :, :-num_wavelet_features]
    wavelet_features = inputs[:, :, -num_wavelet_features:]

    # Main branch
    x_main = Bidirectional(LSTM(units, return_sequences=True))(main_features)
    x_main = ImprovedAttentionLayer()(x_main)
    x_main = Dropout(dropout_rate)(x_main)
    x_main = GRU(64, return_sequences=False)(x_main)
    x_main = Dropout(dropout_rate)(x_main)

    # Wavelet branch
    x_wavelet = Bidirectional(LSTM(units, return_sequences=True))(wavelet_features)
    x_wavelet = ImprovedAttentionLayer()(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)
    x_wavelet = GRU(64, return_sequences=False)(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)

    # Combine branches
    x = Concatenate()([x_main, x_wavelet])
    price_output = Dense(1, activation='linear', name='price')(x)
    class_output = Dense(3, activation='softmax', name='class')(x)
    model = Model(inputs, outputs=[price_output, class_output])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipvalue=0.5)
    model.compile(optimizer=optimizer, 
                  loss={'price': custom_loss, 'class': focal_loss()},
                  metrics={'price': 'mae', 'class': 'accuracy'})
    return model

# TradingEnvironment
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices
        self.trend_labels = np.array(trend_labels)
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward -= 0.05
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward -= 0.05
        else:
            self.hold_count += 1
            reward -= 0.1
        self.actions.append(action)
        self.prev_action = action
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)
        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 100
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 1
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,
            'profit_loss': final_value - self.initial_cash,
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features, num_wavelet_features=5):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.num_wavelet_features = num_wavelet_features
        self.model = create_lstm_model(
            units=64,
            dropout_rate=0.3,
            sequence_length=sequence_length,
            num_features=num_features,
            num_wavelet_features=num_wavelet_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        _, class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices):
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices, label='Normalized Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices[actions == 0]
    sell_prices = prices[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Normalized Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices (no shift)
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Wavelet features (5 features)
def compute_wavelet_features(prices, wavelet='db4', levels=3):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    # Extract cA3, cD3, cD2, cD1, cA2
    cA3, cD3, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2], coeffs[3]
    cA2 = pywt.waverec([coeffs[0], coeffs[1], None, None], wavelet)[:len(prices)]
    # Truncate/pad to match length
    min_len = min(len(cA3), len(cD3), len(cD2), len(cD1), len(cA2), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA3, cD3, cD2, cD1, cA2]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Create labels
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.003:
            labels[i, 0] = 1
        elif diff < -0.003:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 10
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features), num_wavelet_features=5)

# Train LSTM model
agent.model.fit(
    X_train_seq, {'price': y_train_prices, 'class': y_train_seq},
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Data shape after dropna: (3644, 25)
Data shape after upsampling: (5290, 25)
Feature data shape: (5290, 18)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [ 222  230 4838]
X_train_resampled shape: (12054, 18)
y_train_class_resampled shape: (12054, 3)
y_train_resampled shape: (12054,)
Train sequence shape: (12044, 10, 18)
Train labels shape: (12044, 3)
Train prices shape: (12044,)
Test sequence shape: (1048, 10, 18)
Test labels shape: (1048, 3)
Test prices shape: (1048,)
Resampled class distribution: [4018 4018 4008]
Creating LSTM model with units: 64 and dropout rate: 0.3 for sequence length: 10 and num features: 18


2025-04-20 13:20:04.827435: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-20 13:20:04.827637: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-20 13:20:04.827995: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-20 13:20:04.828302: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-20 13:20:04.829038: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2025-04-20 13:20:06.952639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


302/302 ━━━━━━━━━━━━━━━━━━━━ 47s 136ms/step - class_accuracy: 0.5840 - class_loss: 0.0557 - loss: 0.3265 - price_loss: 0.2707 - price_mae: 0.1569 - val_class_accuracy: 0.9149 - val_class_loss: 0.0431 - val_loss: 0.3342 - val_price_loss: 0.2904 - val_price_mae: 0.1745 - learning_rate: 5.0000e-04
Epoch 2/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 38s 126ms/step - class_accuracy: 0.8690 - class_loss: 0.0335 - loss: 0.2734 - price_loss: 0.2399 - price_mae: 0.1492 - val_class_accuracy: 0.9099 - val_class_loss: 0.0320 - val_loss: 0.2716 - val_price_loss: 0.2385 - val_price_mae: 0.1480 - learning_rate: 5.0000e-04
Epoch 3/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 39s 129ms/step - class_accuracy: 0.9114 - class_loss: 0.0252 - loss: 0.2714 - price_loss: 0.2462 - price_mae: 0.1506 - val_class_accuracy: 0.9103 - val_class_loss: 0.0314 - val_loss: 0.2847 - val_price_loss: 0.2521 - val_price_mae: 0.1571 - learning_rate: 5.0000e-04
Epoch 4/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 39s 130ms/step - class_accuracy: 0.9153 - class_lo

In [ ]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional, Concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Custom Loss for Regression
def custom_loss(y_true, y_pred):
    error = tf.abs(y_true - y_pred)
    weight = tf.where(error > 0.05, 5.0, 1.0)
    weighted_mse = tf.reduce_mean(weight * tf.square(y_true - y_pred))
    direction = tf.reduce_mean(tf.sign(y_true[1:] - y_true[:-1]) * tf.sign(y_pred[1:] - y_pred[:-1]))
    drop_penalty = tf.reduce_mean(tf.where(y_true < y_pred, 15.0 * tf.square(y_true - y_pred), 0.0))
    actual_vol = tf.math.reduce_variance(y_true)
    pred_vol = tf.math.reduce_variance(y_pred)
    vol_penalty = tf.square(actual_vol - pred_vol)
    return weighted_mse - 0.1 * direction + 0.5 * drop_penalty + 0.2 * vol_penalty

# Focal Loss for Classification
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Updated LSTM Model with GRU and Wavelet Branch
def create_lstm_model(units, dropout_rate, sequence_length, num_features, num_wavelet_features=5):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    main_features = inputs[:, :, :-num_wavelet_features]
    wavelet_features = inputs[:, :, -num_wavelet_features:]

    # Main branch
    x_main = Bidirectional(LSTM(units, return_sequences=True))(main_features)
    x_main = ImprovedAttentionLayer()(x_main)
    x_main = Dropout(dropout_rate)(x_main)
    x_main = GRU(64, return_sequences=False)(x_main)
    x_main = Dropout(dropout_rate)(x_main)

    # Wavelet branch
    x_wavelet = Bidirectional(LSTM(units, return_sequences=True))(wavelet_features)
    x_wavelet = ImprovedAttentionLayer()(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)
    x_wavelet = GRU(64, return_sequences=False)(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)

    # Combine branches
    x = Concatenate()([x_main, x_wavelet])
    price_output = Dense(1, activation='linear', name='price')(x)
    class_output = Dense(3, activation='softmax', name='class')(x)
    model = Model(inputs, outputs=[price_output, class_output])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipvalue=0.5)
    model.compile(optimizer=optimizer, 
                  loss={'price': custom_loss, 'class': focal_loss()},
                  metrics={'price': 'mae', 'class': 'accuracy'})
    return model

# TradingEnvironment
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices
        self.trend_labels = np.array(trend_labels)
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward -= 0.05
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward -= 0.05
        else:
            self.hold_count += 1
            reward -= 0.1
        self.actions.append(action)
        self.prev_action = action
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)
        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 100
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 1
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,
            'profit_loss': final_value - self.initial_cash,
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features, num_wavelet_features=5):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.num_wavelet_features = num_wavelet_features
        self.model = create_lstm_model(
            units=64,
            dropout_rate=0.3,
            sequence_length=sequence_length,
            num_features=num_features,
            num_wavelet_features=num_wavelet_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        _, class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices):
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices, label='Normalized Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices[actions == 0]
    sell_prices = prices[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Normalized Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices (no shift)
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Wavelet features (5 features)
def compute_wavelet_features(prices, wavelet='db4', levels=3):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    # Extract cA3, cD3, cD2, cD1, cA2
    cA3, cD3, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2], coeffs[3]
    cA2 = pywt.waverec([coeffs[0], coeffs[1], None, None], wavelet)[:len(prices)]
    # Truncate/pad to match length
    min_len = min(len(cA3), len(cD3), len(cD2), len(cD1), len(cA2), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA3, cD3, cD2, cD1, cA2]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Create labels
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.003:
            labels[i, 0] = 1
        elif diff < -0.003:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 10
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features), num_wavelet_features=5)

# Train LSTM model
agent.model.fit(
    X_train_seq, {'price': y_train_prices, 'class': y_train_seq},
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Data shape after dropna: (3644, 25)
Data shape after upsampling: (5290, 25)
Feature data shape: (5290, 18)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [ 222  230 4838]
X_train_resampled shape: (12054, 18)
y_train_class_resampled shape: (12054, 3)
y_train_resampled shape: (12054,)
Train sequence shape: (12044, 10, 18)
Train labels shape: (12044, 3)
Train prices shape: (12044,)
Test sequence shape: (1048, 10, 18)
Test labels shape: (1048, 3)
Test prices shape: (1048,)
Resampled class distribution: [4018 4018 4008]
Creating LSTM model with units: 64 and dropout rate: 0.3 for sequence length: 10 and num features: 18


2025-04-20 13:20:04.827435: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-20 13:20:04.827637: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-20 13:20:04.827995: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-20 13:20:04.828302: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-20 13:20:04.829038: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2025-04-20 13:20:06.952639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


302/302 ━━━━━━━━━━━━━━━━━━━━ 47s 136ms/step - class_accuracy: 0.5840 - class_loss: 0.0557 - loss: 0.3265 - price_loss: 0.2707 - price_mae: 0.1569 - val_class_accuracy: 0.9149 - val_class_loss: 0.0431 - val_loss: 0.3342 - val_price_loss: 0.2904 - val_price_mae: 0.1745 - learning_rate: 5.0000e-04
Epoch 2/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 38s 126ms/step - class_accuracy: 0.8690 - class_loss: 0.0335 - loss: 0.2734 - price_loss: 0.2399 - price_mae: 0.1492 - val_class_accuracy: 0.9099 - val_class_loss: 0.0320 - val_loss: 0.2716 - val_price_loss: 0.2385 - val_price_mae: 0.1480 - learning_rate: 5.0000e-04
Epoch 3/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 39s 129ms/step - class_accuracy: 0.9114 - class_loss: 0.0252 - loss: 0.2714 - price_loss: 0.2462 - price_mae: 0.1506 - val_class_accuracy: 0.9103 - val_class_loss: 0.0314 - val_loss: 0.2847 - val_price_loss: 0.2521 - val_price_mae: 0.1571 - learning_rate: 5.0000e-04
Epoch 4/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 39s 130ms/step - class_accuracy: 0.9153 - class_lo

In [5]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional, Concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Custom Loss for Regression
def custom_loss(y_true, y_pred):
    error = tf.abs(y_true - y_pred)
    weight = tf.where(error > 0.05, 5.0, 1.0)
    weighted_mse = tf.reduce_mean(weight * tf.square(y_true - y_pred))
    direction = tf.reduce_mean(tf.sign(y_true[1:] - y_true[:-1]) * tf.sign(y_pred[1:] - y_pred[:-1]))
    drop_penalty = tf.reduce_mean(tf.where(y_true < y_pred, 15.0 * tf.square(y_true - y_pred), 0.0))
    actual_vol = tf.math.reduce_variance(y_true)
    pred_vol = tf.math.reduce_variance(y_pred)
    vol_penalty = tf.square(actual_vol - pred_vol)
    return weighted_mse - 0.1 * direction + 0.5 * drop_penalty + 0.2 * vol_penalty

# Focal Loss for Classification with Class Weights
def focal_loss(gamma=3.0, alpha=0.5):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_loss = -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
        weights = tf.reduce_sum(y_true * tf.constant([2.0, 2.0, 1.0]), axis=-1)
        return focal_loss * weights
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Updated LSTM Model with GRU and Wavelet Branch
def create_lstm_model(units, dropout_rate, sequence_length, num_features, num_wavelet_features=5):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    main_features = inputs[:, :, :-num_wavelet_features]
    wavelet_features = inputs[:, :, -num_wavelet_features:]

    # Main branch
    x_main = Bidirectional(LSTM(units, return_sequences=True))(main_features)
    x_main = ImprovedAttentionLayer()(x_main)
    x_main = Dropout(dropout_rate)(x_main)
    x_main = GRU(64, return_sequences=False)(x_main)
    x_main = Dropout(dropout_rate)(x_main)

    # Wavelet branch
    x_wavelet = Bidirectional(LSTM(units, return_sequences=True))(wavelet_features)
    x_wavelet = ImprovedAttentionLayer()(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)
    x_wavelet = GRU(64, return_sequences=False)(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)

    # Combine branches
    x = Concatenate()([x_main, x_wavelet])
    price_output = Dense(1, activation='linear', name='price')(x)
    class_output = Dense(3, activation='softmax', name='class')(x)
    model = Model(inputs, outputs=[price_output, class_output])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, 
                  loss={'price': custom_loss, 'class': focal_loss(gamma=3.0, alpha=0.5)},
                  loss_weights={'price': 0.3, 'class': 1.0},
                  metrics={'price': 'mae', 'class': 'accuracy'})
    return model

# TradingEnvironment
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices
        self.trend_labels = np.array(trend_labels)
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward += 0.05  # Encourage BUY
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward += 0.05  # Encourage SELL
        else:
            self.hold_count += 1
            if self.prev_action == 2:  # Consecutive HOLD
                reward -= 0.1
            else:
                reward -= 0.05
        self.actions.append(action)
        self.prev_action = action
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)
        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 100
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 1
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,
            'profit_loss': final_value - self.initial_cash,
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features, num_wavelet_features=5):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.num_wavelet_features = num_wavelet_features
        self.model = create_lstm_model(
            units=128,
            dropout_rate=0.4,
            sequence_length=sequence_length,
            num_features=num_features,
            num_wavelet_features=num_wavelet_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        _, class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices):
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices, label='Normalized Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices[actions == 0]
    sell_prices = prices[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Normalized Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Wavelet features (5 features)
def compute_wavelet_features(prices, wavelet='db4', levels=3):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    cA3, cD3, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2], coeffs[3]
    cA2 = pywt.waverec([coeffs[0], coeffs[1], None, None], wavelet)[:len(prices)]
    min_len = min(len(cA3), len(cD3), len(cD2), len(cD1), len(cA2), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA3, cD3, cD2, cD1, cA2]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA3', 'Wavelet_cD3', 'Wavelet_cD2', 'Wavelet_cD1', 'Wavelet_cA2']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Create labels with adjusted threshold
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.001:
            labels[i, 0] = 1
        elif diff < -0.001:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Apply SMOTE to training data
smote = SMOTE(sampling_strategy={0: 5000, 1: 5000, 2: 5000}, random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 20
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features), num_wavelet_features=5)

# Train LSTM model
agent.model.fit(
    X_train_seq, {'price': y_train_prices, 'class': y_train_seq},
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Data shape after dropna: (3644, 25)
Data shape after upsampling: (5290, 25)
Feature data shape: (5290, 18)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [1102  732 3456]
X_train_resampled shape: (15000, 18)
y_train_class_resampled shape: (15000, 3)
y_train_resampled shape: (15000,)
Train sequence shape: (14980, 20, 18)
Train labels shape: (14980, 3)
Train prices shape: (14980,)
Test sequence shape: (1038, 20, 18)
Test labels shape: (1038, 3)
Test prices shape: (1038,)
Resampled class distribution: [4998 5000 4982]
Creating LSTM model with units: 128 and dropout rate: 0.4 for sequence length: 20 and num features: 18
Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 60s 283ms/step - class_accuracy: 0.4866 - class_loss: 0.0961 - loss: 0.1855 - price_loss: 0.2978 - pri

In [6]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional, Concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Focal Loss for Classification with Class Weights
def focal_loss(gamma=4.0, alpha=0.75):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_loss = -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
        weights = tf.reduce_sum(y_true * tf.constant([3.0, 3.0, 1.0]), axis=-1)
        return focal_loss * weights
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Updated LSTM Model (Classification Only)
def create_lstm_model(units, dropout_rate, sequence_length, num_features, num_wavelet_features=3):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    main_features = inputs[:, :, :-num_wavelet_features]
    wavelet_features = inputs[:, :, -num_wavelet_features:]

    # Main branch
    x_main = Bidirectional(LSTM(units, return_sequences=True))(main_features)
    x_main = ImprovedAttentionLayer()(x_main)
    x_main = Dropout(dropout_rate)(x_main)
    x_main = GRU(64, return_sequences=False)(x_main)
    x_main = Dropout(dropout_rate)(x_main)

    # Wavelet branch
    x_wavelet = Bidirectional(LSTM(units, return_sequences=True))(wavelet_features)
    x_wavelet = ImprovedAttentionLayer()(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)
    x_wavelet = GRU(64, return_sequences=False)(x_wavelet)
    x_wavelet = Dropout(dropout_rate)(x_wavelet)

    # Combine branches
    x = Concatenate()([x_main, x_wavelet])
    class_output = Dense(3, activation='softmax', name='class', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    model = Model(inputs, outputs=class_output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss=focal_loss(gamma=4.0, alpha=0.75), metrics=['accuracy'])
    return model

# TradingEnvironment
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices
        self.trend_labels = np.array(trend_labels)
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward += 0.1
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward += 0.1
        else:
            self.hold_count += 1
            if self.prev_action == 2:
                reward -= 0.3
            else:
                reward -= 0.1
        self.actions.append(action)
        self.prev_action = action
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)
        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 100
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 1
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,
            'profit_loss': final_value - self.initial_cash,
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features, num_wavelet_features=3):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.num_wavelet_features = num_wavelet_features
        self.model = create_lstm_model(
            units=128,
            dropout_rate=0.5,
            sequence_length=sequence_length,
            num_features=num_features,
            num_wavelet_features=num_wavelet_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices):
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices, label='Normalized Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices[actions == 0]
    sell_prices = prices[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Normalized Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Add lagged features
data['Return_Lag1'] = data['Return'].shift(1)
data['Volatility_Lag1'] = data['Volatility'].shift(1)

# Wavelet features (3 features, reduced levels)
def compute_wavelet_features(prices, wavelet='db4', levels=2):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    cA2, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2]
    min_len = min(len(cA2), len(cD2), len(cD1), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA2, cD2, cD1]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1', 'Return_Lag1', 'Volatility_Lag1']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Create labels with adjusted threshold
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.0005:
            labels[i, 0] = 1
        elif diff < -0.0005:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Apply SMOTE to training data
smote = SMOTE(sampling_strategy={0: 4000, 1: 4000, 2: 8000}, random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 15
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features), num_wavelet_features=3)

# Train LSTM model
agent.model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Data shape after dropna: (3644, 25)
Data shape after upsampling: (5290, 25)
Feature data shape: (5290, 18)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [1878 1087 2325]
X_train_resampled shape: (16000, 18)
y_train_class_resampled shape: (16000, 3)
y_train_resampled shape: (16000,)
Train sequence shape: (15985, 15, 18)
Train labels shape: (15985, 3)
Train prices shape: (15985,)
Test sequence shape: (1043, 15, 18)
Test labels shape: (1043, 3)
Test prices shape: (1043,)
Resampled class distribution: [3999 4000 7986]
Creating LSTM model with units: 128 and dropout rate: 0.5 for sequence length: 15 and num features: 18
Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 62s 270ms/step - accuracy: 0.5418 - loss: 0.1682 - val_accuracy: 0.9740 - val_loss: 0.0729 - learning_

In [8]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Focal Loss for Classification with Class Weights
def focal_loss(gamma=3.0, alpha=0.5):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_loss = -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
        weights = tf.reduce_sum(y_true * tf.constant([2.0, 2.0, 1.0]), axis=-1)
        return focal_loss * weights
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Simplified LSTM Model (Single Branch)
def create_lstm_model(units, dropout_rate, sequence_length, num_features):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    x = Bidirectional(LSTM(units, return_sequences=True))(inputs)
    x = ImprovedAttentionLayer()(x)
    x = Dropout(dropout_rate)(x)
    x = GRU(64, return_sequences=False)(x)
    x = Dropout(dropout_rate)(x)
    class_output = Dense(3, activation='softmax', name='class', kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    model = Model(inputs, outputs=class_output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss=focal_loss(gamma=3.0, alpha=0.5), metrics=['accuracy'])
    return model

# TradingEnvironment
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices
        self.trend_labels = np.array(trend_labels)
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward += 0.05
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward += 0.05
        else:
            self.hold_count += 1
            if self.prev_action == 2:
                reward -= 0.1
            else:
                reward -= 0.05
        self.actions.append(action)
        self.prev_action = action
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)
        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 200
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 0.5
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,
            'profit_loss': final_value - self.initial_cash,
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.model = create_lstm_model(
            units=128,
            dropout_rate=0.5,
            sequence_length=sequence_length,
            num_features=num_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices):
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices, label='Normalized Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices[actions == 0]
    sell_prices = prices[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Normalized Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Normalized Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Add lagged features
data['Return_Lag1'] = data['Return'].shift(1)
data['Volatility_Lag1'] = data['Volatility'].shift(1)
data['Return_Lag2'] = data['Return'].shift(2)
data['Volatility_Lag2'] = data['Volatility'].shift(2)
data['VIX'] = data['Volatility'].rolling(window=20).mean()  # Placeholder for VIX

# Wavelet features (3 features)
def compute_wavelet_features(prices, wavelet='db4', levels=2):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    cA2, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2]
    min_len = min(len(cA2), len(cD2), len(cD1), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA2, cD2, cD1]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1', 'Return_Lag1', 'Volatility_Lag1',
            'Return_Lag2', 'Volatility_Lag2', 'VIX']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Create labels with adjusted threshold
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.006:
            labels[i, 0] = 1
        elif diff < -0.006:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Debug: Test set class distribution
print("Test set class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_test_class, axis=1)))

# Apply SMOTE to training data
smote = SMOTE(sampling_strategy={0: 2000, 1: 2000, 2: 8000}, random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 20
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features))

# Train LSTM model
agent.model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Data shape after dropna: (3644, 28)
Data shape after upsampling: (5290, 28)
Feature data shape: (5290, 21)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [  34   72 5184]
Test set class distribution (UP, DOWN, NEUTRAL): [  22   36 1000]
X_train_resampled shape: (12000, 21)
y_train_class_resampled shape: (12000, 3)
y_train_resampled shape: (12000,)
Train sequence shape: (11980, 20, 21)
Train labels shape: (11980, 3)
Train prices shape: (11980,)
Test sequence shape: (1038, 20, 21)
Test labels shape: (1038, 3)
Test prices shape: (1038,)
Resampled class distribution: [2000 2000 7980]
Creating LSTM model with units: 128 and dropout rate: 0.5 for sequence length: 20 and num features: 21
Epoch 1/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 23s 124ms/step - accuracy: 0.7318 - 

In [11]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Focal Loss for Classification with Class Weights
def focal_loss(gamma=3.0, alpha=0.5):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_loss = -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
        weights = tf.reduce_sum(y_true * tf.constant([2.0, 2.0, 1.0]), axis=-1)
        return focal_loss * weights
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Simplified LSTM Model (Single Branch)
def create_lstm_model(units, dropout_rate, sequence_length, num_features):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    x = Bidirectional(LSTM(units, return_sequences=True))(inputs)
    x = ImprovedAttentionLayer()(x)
    x = Dropout(dropout_rate)(x)
    x = GRU(64, return_sequences=False)(x)
    x = Dropout(dropout_rate)(x)
    class_output = Dense(3, activation='softmax', name='class', kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    model = Model(inputs, outputs=class_output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss=focal_loss(gamma=3.0, alpha=0.5), metrics=['accuracy'])
    return model

# TradingEnvironment (Updated to Denormalize Portfolio Value)
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, scaler_y, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices  # Normalized prices
        self.trend_labels = np.array(trend_labels)
        self.scaler_y = scaler_y  # Scaler to denormalize prices
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        # Denormalize initial_cash based on the price scale
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        # Denormalize the current price
        current_price_normalized = self.prices[self.current_step]
        current_price = self.scaler_y.inverse_transform([[current_price_normalized]])[0][0]

        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward += 0.05
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward += 0.05
        else:
            self.hold_count += 1
            if self.prev_action == 2:
                reward -= 0.1
            else:
                reward -= 0.05
        self.actions.append(action)
        self.prev_action = action

        # Compute portfolio value in the original scale
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)

        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 200
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 0.5
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,  # Now in original scale
            'profit_loss': final_value - self.initial_cash,  # Now in original scale
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.model = create_lstm_model(
            units=128,
            dropout_rate=0.5,
            sequence_length=sequence_length,
            num_features=num_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices, scaler_y):
    # Denormalize prices for plotting
    prices_denorm = scaler_y.inverse_transform(prices.reshape(-1, 1)).flatten()
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices_denorm, label='Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices_denorm[actions == 0]
    sell_prices = prices_denorm[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Price (Original Scale)')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Add lagged features
data['Return_Lag1'] = data['Return'].shift(1)
data['Volatility_Lag1'] = data['Volatility'].shift(1)
data['Return_Lag2'] = data['Return'].shift(2)
data['Volatility_Lag2'] = data['Volatility'].shift(2)
data['VIX'] = data['Volatility'].rolling(window=20).mean()  # Placeholder for VIX

# Wavelet features (3 features)
def compute_wavelet_features(prices, wavelet='db4', levels=2):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    cA2, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2]
    min_len = min(len(cA2), len(cD2), len(cD1), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA2, cD2, cD1]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1', 'Return_Lag1', 'Volatility_Lag1',
            'Return_Lag2', 'Volatility_Lag2', 'VIX']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Debug: Print min and max of Target for denormalization reference
print(f"Target min (original): {scaler_y.data_min_[0]}, Target max (original): {scaler_y.data_max_[0]}")

# Create labels with adjusted threshold
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.005:
            labels[i, 0] = 1
        elif diff < -0.005:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Debug: Test set class distribution
print("Test set class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_test_class, axis=1)))

# Apply SMOTE to training data
smote = SMOTE(sampling_strategy={0: 2000, 1: 2000, 2: 8000}, random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 20
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features))

# Train LSTM model
agent.model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment with scaler_y
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, scaler_y=scaler_y, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals (denormalized)
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices, scaler_y)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Target min (original): 1040.0108244311277, Target max (original): 6111.065995579977
Data shape after dropna: (3644, 28)
Data shape after upsampling: (5290, 28)
Feature data shape: (5290, 21)
Prices shape: (5290,)
Trend labels shape: (5290, 3)
Class distribution (UP, DOWN, NEUTRAL): [  60  106 5124]
Test set class distribution (UP, DOWN, NEUTRAL): [ 39  53 966]
X_train_resampled shape: (12000, 21)
y_train_class_resampled shape: (12000, 3)
y_train_resampled shape: (12000,)
Train sequence shape: (11980, 20, 21)
Train labels shape: (11980, 3)
Train prices shape: (11980,)
Test sequence shape: (1038, 20, 21)
Test labels shape: (1038, 3)
Test prices shape: (1038,)
Resampled class distribution: [2000 2000 7980]
Creating LSTM model with units: 128 and dropout rate: 0.5 for sequence length: 20 and num features

In [15]:
import gym
import numpy as np
from gym import spaces
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import yfinance as yf
from ta.volatility import BollingerBands
from pykalman import KalmanFilter
import pywt
import time

# Focal Loss for Classification with Class Weights
def focal_loss(gamma=3.0, alpha=0.5):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_loss = -tf.reduce_mean(alpha_t * tf.pow(1.0 - pt, gamma) * tf.math.log(pt))
        weights = tf.reduce_sum(y_true * tf.constant([2.0, 2.0, 1.0]), axis=-1)
        return focal_loss * weights
    return focal_loss_fixed

# Improved Attention Layer
class ImprovedAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads=4):
        super(ImprovedAttentionLayer, self).__init__()
        self.num_heads = num_heads

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model % self.num_heads == 0
        self.depth = self.d_model // self.num_heads
        self.Wq = self.add_weight(name='Wq', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wk = self.add_weight(name='Wk', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wv = self.add_weight(name='Wv', shape=(self.d_model, self.d_model), initializer='glorot_uniform')
        self.Wo = self.add_weight(name='Wo', shape=(self.d_model, self.d_model), initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        Q = tf.matmul(inputs, self.Wq)
        K = tf.matmul(inputs, self.Wk)
        V = tf.matmul(inputs, self.Wv)
        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)
        dk = tf.cast(self.depth, tf.float32)
        score = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(attention_weights, V)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        context = tf.reshape(context, (batch_size, -1, self.d_model))
        output = tf.matmul(context, self.Wo)
        return output

# Simplified LSTM Model (Single Branch)
def create_lstm_model(units, dropout_rate, sequence_length, num_features):
    print("Creating LSTM model with units:", units, "and dropout rate:", dropout_rate,
          "for sequence length:", sequence_length, "and num features:", num_features)
    inputs = Input(shape=(sequence_length, num_features))
    x = Bidirectional(LSTM(units, return_sequences=True))(inputs)
    x = ImprovedAttentionLayer()(x)
    x = Dropout(dropout_rate)(x)
    x = GRU(64, return_sequences=False)(x)
    x = Dropout(dropout_rate)(x)
    class_output = Dense(3, activation='softmax', name='class', kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    model = Model(inputs, outputs=class_output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss=focal_loss(gamma=3.0, alpha=0.5), metrics=['accuracy'])
    return model

# TradingEnvironment (Updated to Denormalize Portfolio Value)
class TradingEnvironment(gym.Env):
    def __init__(self, data, features, prices, trend_labels, scaler_y, initial_cash=10000, max_steps=1000):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.features = features
        self.prices = prices  # Normalized prices
        self.trend_labels = np.array(trend_labels)
        self.scaler_y = scaler_y  # Scaler to denormalize prices
        self.initial_cash = initial_cash
        self.max_steps = max_steps
        self.current_step = 0
        # Denormalize initial_cash based on the price scale
        self.cash = initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(features),),
            dtype=np.float32
        )
        self.feature_means = np.mean(data, axis=0)
        self.feature_stds = np.std(data, axis=0) + 1e-6
        self.normalized_data = (data - self.feature_means) / self.feature_stds
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        print(f"Data shape: {self.data.shape}, Prices shape: {self.prices.shape}, Labels shape: {self.trend_labels.shape}")
        print(f"Observation space shape: {self.observation_space.shape}")

    def reset(self):
        self.current_step = 0
        self.cash = self.initial_cash
        self.holdings = 0
        self.portfolio_values = []
        self.actions = []
        self.buy_count = 0
        self.sell_count = 0
        self.hold_count = 0
        self.prev_action = 2
        return self._get_observation()

    def _get_observation(self):
        if self.current_step >= len(self.data):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        state = self.normalized_data[self.current_step]
        return state.astype(np.float32)

    def step(self, action):
        # Denormalize the current price
        current_price_normalized = self.prices[self.current_step]
        current_price = self.scaler_y.inverse_transform([[current_price_normalized]])[0][0]

        reward = 0
        if action == 0 and self.cash >= current_price:
            self.holdings += 1
            self.cash -= current_price
            self.buy_count += 1
            reward += 0.05
        elif action == 1 and self.holdings > 0:
            self.holdings -= 1
            self.cash += current_price
            self.sell_count += 1
            reward += 0.05
        else:
            self.hold_count += 1
            if self.prev_action == 2:
                reward -= 0.1
            else:
                reward -= 0.05
        self.actions.append(action)
        self.prev_action = action

        # Compute portfolio value in the original scale
        portfolio_value = self.cash + self.holdings * current_price
        self.portfolio_values.append(portfolio_value)

        if len(self.portfolio_values) > 1:
            portfolio_change = (portfolio_value - self.portfolio_values[-2]) / self.initial_cash
            reward += portfolio_change * 200
            if len(self.portfolio_values) >= 11:
                returns = np.diff(self.portfolio_values[-10:]) / self.portfolio_values[-11:-2]
                volatility = np.std(returns)
                reward -= volatility * 0.5
        self.current_step += 1
        done = self.current_step >= min(len(self.data), self.max_steps)
        next_state = self._get_observation() if not done else np.zeros_like(self._get_observation())
        info = {'portfolio_value': portfolio_value}
        return next_state, reward, done, info

    def get_metrics(self):
        total_steps = len(self.actions)
        predicted_labels = np.array(self.actions)
        true_labels = np.argmax(self.trend_labels[:total_steps], axis=1)
        accuracy = np.mean(predicted_labels == true_labels) if total_steps > 0 else 0
        final_value = self.portfolio_values[-1] if self.portfolio_values else self.initial_cash
        sharpe = 0
        if len(self.portfolio_values) > 1:
            returns = np.diff(self.portfolio_values) / np.array(self.portfolio_values[:-1])
            if len(returns) > 0 and np.std(returns) > 0:
                sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
        return {
            'final_portfolio_value': final_value,  # Now in original scale
            'profit_loss': final_value - self.initial_cash,  # Now in original scale
            'sharpe_ratio': sharpe,
            'buy_count': self.buy_count,
            'sell_count': self.sell_count,
            'hold_count': self.hold_count,
            'action_accuracy': accuracy,
            'predicted_labels': predicted_labels,
            'true_labels': true_labels
        }

# LSTM Agent
class LSTMAgent:
    def __init__(self, sequence_length, num_features):
        self.sequence_length = sequence_length
        self.num_features = num_features
        self.model = create_lstm_model(
            units=128,
            dropout_rate=0.5,
            sequence_length=sequence_length,
            num_features=num_features
        )

    def prepare_sequence(self, data, current_step):
        if current_step < self.sequence_length:
            return np.zeros((self.sequence_length, self.num_features), dtype=np.float32)
        sequence = data[max(0, current_step - self.sequence_length):current_step]
        if len(sequence) < self.sequence_length:
            padding = np.zeros((self.sequence_length - len(sequence), self.num_features), dtype=np.float32)
            sequence = np.vstack([padding, sequence])
        return sequence

    def act(self, sequence):
        sequence = np.expand_dims(sequence, axis=0)
        class_probs = self.model.predict(sequence, verbose=0)
        return np.argmax(class_probs[0])

# Plotting functions
def plot_confusion_matrix(true_labels, predicted_labels, classes=['UP', 'DOWN', 'NEUTRAL']):
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (LSTM)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix_lstm.png')
    plt.close()

def plot_prices(prices, actions, indices, scaler_y):
    # Denormalize prices for plotting
    prices_denorm = scaler_y.inverse_transform(prices.reshape(-1, 1)).flatten()
    plt.figure(figsize=(12, 6))
    plt.plot(indices, prices_denorm, label='Price (Target)', color='blue')
    buy_signals = indices[actions == 0]
    sell_signals = indices[actions == 1]
    buy_prices = prices_denorm[actions == 0]
    sell_prices = prices_denorm[actions == 1]
    plt.scatter(buy_signals, buy_prices, color='green', marker='^', label='Buy', alpha=0.6)
    plt.scatter(sell_signals, sell_prices, color='red', marker='v', label='Sell', alpha=0.6)
    plt.title('Price with Buy/Sell Signals (LSTM)')
    plt.xlabel('Time Step')
    plt.ylabel('Price (Original Scale)')
    plt.legend()
    plt.grid(True)
    plt.savefig('price_plot_lstm.png')
    plt.close()

# Data preparation
data = yf.download('^GSPC', start='2010-01-01', end='2025-04-12')

# Validate date range
print(f"Data date range: {data.index[0]} to {data.index[-1]}")
print(f"Raw data shape: {data.shape}")

# Flatten MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    print("MultiIndex detected, flattening columns...")
    data.columns = [col[0] for col in data.columns]
print("Initial columns:", data.columns.tolist())

# Apply Kalman Filter to smooth Close prices
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=data['Close'].iloc[0],
    initial_state_covariance=1.0,
    observation_covariance=1.0,
    transition_covariance=0.1
)
smoothed_prices, _ = kf.filter(data['Close'].shift(-3).values)
data['Target'] = smoothed_prices

# Compute indicators
data['SMA_crossover'] = data['Close'].rolling(window=200).mean() - data['Close'].rolling(window=50).mean()

def average_true_range(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = true_range.rolling(window).mean()
    return df

data = average_true_range(data)

def on_balance_volume(df):
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    return df

data = on_balance_volume(data)

def vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

data = vwap(data)

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

ema_12 = data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_12 - ema_26

data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(window=10).std()

bb_indicator = BollingerBands(close=data['Close'], window=20, window_dev=2)
data['BB_bandWidth'] = bb_indicator.bollinger_hband() - bb_indicator.bollinger_lband()

data['SMA_17'] = data['Close'].rolling(window=17).mean()
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_9'] = data['Close'].rolling(window=9).mean()

data['MACD_Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']

# Add lagged features
data['Return_Lag1'] = data['Return'].shift(1)
data['Volatility_Lag1'] = data['Volatility'].shift(1)
data['Return_Lag2'] = data['Return'].shift(2)
data['Volatility_Lag2'] = data['Volatility'].shift(2)
data['VIX'] = data['Volatility'].rolling(window=20).mean()  # Placeholder for VIX

# Wavelet features (3 features)
def compute_wavelet_features(prices, wavelet='db4', levels=2):
    coeffs = pywt.wavedec(prices, wavelet, level=levels, mode='smooth')
    cA2, cD2, cD1 = coeffs[0], coeffs[1], coeffs[2]
    min_len = min(len(cA2), len(cD2), len(cD1), len(prices))
    return [np.pad(c[:min_len], (0, len(prices) - min_len), mode='constant') for c in [cA2, cD2, cD1]]

wavelet_features = compute_wavelet_features(data['Target'].values)
for i, name in enumerate(['Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1']):
    data[name] = wavelet_features[i]

# Normalize features and target
features = ['SMA_crossover', 'SMA_17', 'SMA_9', 'SMA_5', 'ATR_14', 'OBV',
            'VWAP', 'Volume', 'BB_bandWidth', 'MACD', 'RSI_14', 'MACD_Histogram', 'Volatility',
            'Wavelet_cA2', 'Wavelet_cD2', 'Wavelet_cD1', 'Return_Lag1', 'Volatility_Lag1',
            'Return_Lag2', 'Volatility_Lag2', 'VIX']
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
data[features] = scaler_X.fit_transform(data[features])
data['Target'] = scaler_y.fit_transform(data['Target'].values.reshape(-1, 1))

# Debug: Print min and max of Target for denormalization reference
print(f"Target min (original): {scaler_y.data_min_[0]}, Target max (original): {scaler_y.data_max_[0]}")

# Create labels with adjusted threshold
def create_labels(y):
    labels = np.zeros((len(y), 3))
    for i in range(len(y) - 1):
        diff = y[i + 1] - y[i]
        if diff > 0.005:
            labels[i, 0] = 1
        elif diff < -0.005:
            labels[i, 1] = 1
        else:
            labels[i, 2] = 1
    labels[-1, 2] = 1
    return labels

# Clean data
data.dropna(inplace=True)
print(f"Data shape after dropna: {data.shape}")

# Upsample to 6915 rows (placeholder)
if len(data) < 6915:
    data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq='D'))
    data = data.interpolate(method='linear')
    data = data.dropna()
    print(f"Data shape after upsampling: {data.shape}")

X = data[features].values
y = data['Target'].values
y_class = create_labels(y)

# Debug: Verify shapes and class distribution
print("Feature data shape:", X.shape)
print("Prices shape:", y.shape)
print("Trend labels shape:", y_class.shape)
print("Class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_class, axis=1)))

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
y_train_class, y_test_class = y_class[:split], y_class[split:]

# Debug: Test set class distribution
print("Test set class distribution (UP, DOWN, NEUTRAL):", np.bincount(np.argmax(y_test_class, axis=1)))

# Apply SMOTE to training data
smote = SMOTE(sampling_strategy={0: 2000, 1: 2000, 2: 8000}, random_state=42)
X_train_flat = X_train
y_train_class_flat = np.argmax(y_train_class, axis=1)
X_train_resampled, y_train_class_resampled = smote.fit_resample(X_train_flat, y_train_class_flat)
y_train_class_resampled = np.eye(3)[y_train_class_resampled]

# Align y_train_resampled with X_train_resampled
n_samples = len(X_train_resampled)
y_train_resampled = np.zeros(n_samples)
indices = np.arange(len(y_train))
sampled_indices = np.random.choice(indices, size=n_samples, replace=True)
y_train_resampled = y_train[sampled_indices]

# Debug: Verify resampled shapes
print("X_train_resampled shape:", X_train_resampled.shape)
print("y_train_class_resampled shape:", y_train_class_resampled.shape)
print("y_train_resampled shape:", y_train_resampled.shape)

# Prepare sequences for LSTM
lookback = 20
def create_sequences(data, labels, prices, lookback):
    if not (len(data) == len(labels) == len(prices)):
        raise ValueError(f"Length mismatch: data={len(data)}, labels={len(labels)}, prices={len(prices)}")
    n_samples = min(len(data), len(labels), len(prices))
    sequences = []
    seq_labels = []
    seq_prices = []
    for i in range(lookback, n_samples):
        sequences.append(data[i-lookback:i])
        seq_labels.append(labels[i])
        seq_prices.append(prices[i])
    return np.array(sequences), np.array(seq_labels), np.array(seq_prices)

# Create sequences
X_train_seq, y_train_seq, y_train_prices = create_sequences(X_train_resampled, y_train_class_resampled, y_train_resampled, lookback)
X_test_seq, y_test_seq, y_test_prices = create_sequences(X_test, y_test_class, y_test, lookback)

# Debug: Verify sequence shapes
print("Train sequence shape:", X_train_seq.shape)
print("Train labels shape:", y_train_seq.shape)
print("Train prices shape:", y_train_prices.shape)
print("Test sequence shape:", X_test_seq.shape)
print("Test labels shape:", y_test_seq.shape)
print("Test prices shape:", y_test_prices.shape)
print("Resampled class distribution:", np.bincount(np.argmax(y_train_seq, axis=1)))

# Initialize LSTM agent
agent = LSTMAgent(sequence_length=lookback, num_features=len(features))

# Train LSTM model
agent.model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ],
    verbose=1
)

# Initialize test environment with scaler_y
test_env = TradingEnvironment(X_test, features, y_test, y_test_class, scaler_y=scaler_y, max_steps=len(y_test))

# Testing phase
print("\nTesting...")
state = test_env.reset()
total_reward = 0
start_time = time.time()
for step in range(lookback, len(X_test)):
    sequence = test_env.normalized_data[step-lookback:step]
    if len(sequence) < lookback:
        sequence = np.pad(sequence, ((lookback-len(sequence), 0), (0, 0)), mode='constant')
    action = agent.act(sequence)
    next_state, reward, done, info = test_env.step(action)
    state = next_state
    total_reward += reward
    if done:
        break
metrics = test_env.get_metrics()
print(f"Test Results: Total Reward: {total_reward:.2f}, "
      f"Portfolio Value: {metrics['final_portfolio_value']:.2f}, "
      f"Profit/Loss: {metrics['profit_loss']:.2f}, "
      f"Sharpe: {metrics['sharpe_ratio']:.2f}, "
      f"Accuracy: {metrics['action_accuracy']:.2f}, "
      f"Buy Count: {metrics['buy_count']}, "
      f"Sell Count: {metrics['sell_count']}, "
      f"Hold Count: {metrics['hold_count']}, "
      f"Time: {time.time() - start_time:.2f}s")

# Plot confusion matrix
plot_confusion_matrix(metrics['true_labels'], metrics['predicted_labels'])

# Plot prices with buy/sell signals (denormalized)
test_indices = np.arange(len(metrics['predicted_labels']))
plot_prices(y_test[lookback:lookback+len(metrics['predicted_labels'])], metrics['predicted_labels'], test_indices, scaler_y)

[*********************100%***********************]  1 of 1 completed


Data date range: 2010-01-04 00:00:00 to 2025-04-11 00:00:00
Raw data shape: (3843, 5)
MultiIndex detected, flattening columns...
Initial columns: ['Close', 'High', 'Low', 'Open', 'Volume']
Target min (original): 1040.0108244311277, Target max (original): 6111.065995579977
Data shape after dropna: (3635, 28)
Data shape after upsampling: (5287, 28)
Feature data shape: (5287, 21)
Prices shape: (5287,)
Trend labels shape: (5287, 3)
Class distribution (UP, DOWN, NEUTRAL): [  65  101 5121]
Test set class distribution (UP, DOWN, NEUTRAL): [ 38  45 975]
X_train_resampled shape: (12000, 21)
y_train_class_resampled shape: (12000, 3)
y_train_resampled shape: (12000,)
Train sequence shape: (11980, 20, 21)
Train labels shape: (11980, 3)
Train prices shape: (11980,)
Test sequence shape: (1038, 20, 21)
Test labels shape: (1038, 3)
Test prices shape: (1038,)
Resampled class distribution: [2000 2000 7980]
Creating LSTM model with units: 128 and dropout rate: 0.5 for sequence length: 20 and num features